In [1]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset

from sklearn.metrics import classification_report

In [ ]:
dataset = load_dataset("tweet_eval", "sentiment")
print(dataset)

KeyboardInterrupt: 

In [ ]:
type(dataset)